<h1 style="background-color:#FFC300;"></h1>
<center><img src="./BCCC_banner_814.png" alt="Header" style="width: 814px;"/></center>
<h1 style="background-color:#FFC300;"></h1>

# Wispa, Wispa Gold or Crunchie ?!
### Machine-Learning Computer-Vision Recognition Outreach Project for the Materials Made Smarter Centre adapted to support the Discover Materials Chocolate Impacter!

This is an adaptation of the the Bottle, Can or Coffee Cup project described bellow with categories defined for recognising a Wispa bar, Wispa Gold bar or Crunchie Bar.

This adaptation was created to demonstrate the alternative uses of the original outreach project live at the Festival of Tomorrow 21st and 22nd February 2025 whilst on the Discover Materials stand.

The event is documented at https://discovermaterials.co.uk/news/discover-materials-ambassadors-at-the-festival-of-tomorrow/


This project has been developed by the Materials Made Smarter Centre at Swansea University in collaboration with the Sustain Manufacturing Research Hub and Discover Materials to demonstrate how Computer Vision and Machine Learning can be used to recognise different objects to help with the sorting of materials for recycling.

Further information and documentation about this project can be found at https://discovermaterials.co.uk/resource/bottle-can-or-coffee-cup/

<center><img src="./Equipment_setup_814.jpg" alt="Equipment" style="width: 814px;"/></center>

The Project Equipment, a portable monitor and two combined Seeed reComputer J1010 units hosting the NVIDIA Jetson Nano Jetpak project code

The platform this project is built on is the Seeed Studio reComputer J1010 NVIDIA Jetson Nano 2GB Platform with the Arm Cortex A57 CPU and NVIDIA Maxwell GPU and it has been developed by Dr R. Gibbs and Prof. C. Giannetti based upon the NVIDIA DLI "Getting Started with AI on Jetson Nano” course which can be found in the ../classification directory
 
Professor C. Giannetti would like to acknowledge the support of the EPSRC (EP/V061798/1) in this Materials Made Smarter Project.

<h1 style="background-color:#FFC300;"></h1>

### Launch Camera
This cell opens access to the Logitech C270 webcam attached to the USB3 port on reComputer 1. The jetcam library is contained within the nvidia_dli_docker environment.

In [ ]:
## Launch Camera ################################################################################

# Check device number
!ls -ltrh /dev/video*

from jetcam.usb_camera import USBCamera

# Logitech C270 webcam
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera started")

################################################################################################

### Define Machine-Learning Task
The task will be a Classification rather than a Regression Machine-Learning task. Classifying what the camera sees as either a bottle, a can or a coffee cup. There are three Datasets A, B or C. Dataset A contains 50 images each of the example props for the project. Datasets B and C can be used for other examples of training for new images. The images captured for training are stored in, for example, ./images/BCCC_A/Bottle/ etc.

In [ ]:
## Define Machine Learning Task ################################################################

import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'WWGC'

CATEGORIES = ['Wispa', 'Wispa_Gold', 'Crunchie']

DATASETS = ['A', 'B', 'C']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('../data/images/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

# Set up the data directory location if not there already
DATA_DIR = '/nvdli-nano/data/images/'
!mkdir -p {DATA_DIR}

################################################################################################

### Create Widgets used for Data Collection
Creates the widgets that display the dataset and categories being collected, the image from the camera and the capture button which stores an image

In [ ]:
## Create Data Collection Widget ################################################################

import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image from camera
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='Dataset:')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='Category:')
count_widget = ipywidgets.IntText(description='# Images:')
capture_widget = ipywidgets.Button(description='Capture Image')

# update exisitng count of images at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
capture_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([capture_widget,
                                          dataset_widget,
                                          category_widget,
                                          count_widget])

# output
print("camera_widget, data_collection_widget created")

################################################################################################

### Load the pre-trained RESNET 18 Neural Network Model
There are several large Neural Network models which have been pre-trained on millions of general images to develop general understanding of what the camera is looking at. Through transfer learning, these pretrained models are used as the starting point from which new training is performed to specialise the recognition to a few specific categories. This permits successful models to be built using fewer training images than would be required if trying to learn a situation from scratch. It takes a short time to load in the model that is being used. Three other models are also available for experiementation. The models are stored in the nvidia_dli_docker environment.

In [ ]:
################################################################################################

import torch
import torchvision

device = torch.device('cuda') # this makes use of the Graphical Processing Unit built into the Jetson Nano.

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(dataset.categories))

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, len(dataset.categories))

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, len(dataset.categories), kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, len(dataset.categories))
    
model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='/nvdli-nano/data/WWGC_model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.HBox([model_path_widget,model_load_button,model_save_button])

# output
print("model_widget created")

################################################################################################

### Create State and Prediction Widgets
Switch between Training and Prediction states and launch the live real-time state. In the real-time state the system displays a live view of what the camera sees, but waits until the 'recognise' button is pressed before performing a prediction/inference operation with the trained model.

In [ ]:
################################################################################################

import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['Train', 'Predict'], description='State', value='Train')
prediction_widget = ipywidgets.Text(description='as') # this widget is placed after the 'Recognise' widget so that it reads Recognise as ...
score_widgets = []
for category in dataset.categories:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)
    
recognise_widget = ipywidgets.Button(description='Recognise')

# run prediction on current image
def recognise(c):
    image = camera.value
    preprocessed = preprocess(image)
    output = model(preprocessed)
    output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
    category_index = output.argmax()
    prediction_widget.value = dataset.categories[category_index]
    for i, score in enumerate(list(output)):
        score_widgets[i].value = score

# during the live state the camera feed thread is run in real-time but and the 'Recognise' button is polled
# inference is only performed by the recognise function when the button is pressed
def live(state_widget, model, camera, prediction_widget, score_widget):
    global dataset
    while state_widget.value == 'Predict':
        recognise_widget.on_click(recognise) 
                        
def start_live(change):
    if change['new'] == 'Predict':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread.start()
state_widget.observe(start_live, names='value')

predict_widget = ipywidgets.VBox([ipywidgets.HBox(score_widgets),
                                  ipywidgets.HBox([recognise_widget,prediction_widget])])

# outputs
print("state_widget and predict_widget created")

################################################################################################

### Training and Evaluation
Define the training widgets, after new data is collected the final layer of the model must be retrained to learn the new images. This cell may take several seconds to execute. The default number of epochs for training the model is 10. The first epoch takes a while to begin as the images are loaded into memmory, but then training proceeds relatively quickly, counting epochs down. Once training is complete the system automatically switches to the 'Predict' state and waits for the 'recognise' button to be pressed.

In [ ]:
################################################################################################

BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=10)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
accuracy_widget = ipywidgets.FloatText(description='accuracy')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'Train'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'Predict'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

training_widget = ipywidgets.VBox([ipywidgets.HBox([train_button,eval_button]),
                                   epochs_widget,
                                   progress_widget,
                                   accuracy_widget]) 

# loss_widget not included

# output
print("training configured and training_widget created")

################################################################################################

### Define Project Controls
Groups and arranges all the created widgets into the display for the project

In [ ]:
## define project controls #################################################################
# constructed from:
# camera_widget, data_collection_widget
# modelsave_widget
# state_widget, predict_widget and score_widgets
# training_widget
project_controls = ipywidgets.VBox([state_widget,
                                    ipywidgets.HBox([camera_widget,predict_widget]),
                                    ipywidgets.HBox([data_collection_widget,training_widget]),
                                    model_widget])

### Launch the Project Controls

<h1 style="background-color:#FFC300;"></h1>
<center><img src="./BCCC_banner_814.png" alt="Header" style="width: 814px;"/></center>
<h1 style="background-color:#FFC300;"></h1>

In [ ]:
display(project_controls)

<h1 style="background-color:#FFC300;"></h1>